In [16]:
# Import all needed libraries
#! pip install google-cloud-bigquery
from google.cloud import bigquery
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
#from config import db_password

In [17]:
# GCP Authentication
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="client_secret.json"

In [18]:
client = bigquery.Client()

In [19]:
# Exploring raw dataset
hn_dataset_ref = client.dataset('stackoverflow', project='bigquery-public-data')
hn_dataset_ref

DatasetReference('bigquery-public-data', 'stackoverflow')

In [20]:
type(hn_dataset_ref)

google.cloud.bigquery.dataset.DatasetReference

In [21]:
hn_dset = client.get_dataset(hn_dataset_ref)
hn_dset

Dataset(DatasetReference('bigquery-public-data', 'stackoverflow'))

In [22]:
type(hn_dset)

google.cloud.bigquery.dataset.Dataset

In [23]:
# Determine available data tables
[x.table_id for x in client.list_tables(hn_dset)]

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [24]:
# Exploring data in posts_questions
hn_full = client.get_table(table='bigquery-public-data.stackoverflow.posts_questions')
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-01-01'
        ORDER BY accepted_answer_id
        LIMIT 10"""
    )

results = query_job.result() 
results_list = [row for row in results]
cols = [field.name for field in results.schema]
posts_questions = pd.DataFrame.from_records(results_list, columns=cols)
posts_questions

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,65526420,How to check if values in individiual rows of ...,<p>Suppose I have the following data.table:</p...,65526457,2,2,None,2021-01-01 00:05:46.310000+00:00,2.0,2021-01-01 02:15:42.223000+00:00,NaT,None,NaN,None,6335637,None,1,2,r|data.table|rowwise,62
1,65526423,d3.js : Generating axis ticks for ordinal values,<p>I want to use ordinal scale in x-axis with ...,65526533,1,0,None,2021-01-01 00:06:09.007000+00:00,NaN,2021-01-01 16:25:58.447000+00:00,NaT,None,NaN,None,9574155,None,1,2,javascript|d3.js,48
2,65526490,Is there a C macro that replaces varied length...,<p>I want to be able to:</p>\n<pre><code>#defi...,65526541,1,1,None,2021-01-01 00:20:09.553000+00:00,0.0,2021-01-01 00:37:24.277000+00:00,NaT,None,NaN,None,12743240,None,1,2,c|macros,35
3,65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,65526554,2,0,None,2021-01-01 00:05:43.627000+00:00,NaN,2021-01-01 18:19:46.870000+00:00,2021-01-01 05:23:28.560000+00:00,None,65863.0,None,14694500,None,1,3,c++|std|c++20|allocator,351
4,65526523,Navigate from parent to child in react where e...,<p>In a React app with a parent and child elem...,65526577,1,1,None,2021-01-01 00:30:31.933000+00:00,NaN,2021-01-01 00:46:22.813000+00:00,2021-01-01 00:35:53.623000+00:00,None,8690857.0,None,3002584,None,1,2,reactjs|react-router|react-router-dom,117
5,65526451,How do I define a field in a subclass by stric...,<p>I'm trying to implement the UML diagram bel...,65526589,1,0,None,2021-01-01 00:11:17.787000+00:00,NaN,2021-01-01 10:22:51.790000+00:00,2021-01-01 10:22:51.790000+00:00,None,2458991.0,None,14634129,None,1,1,java|inheritance|uml,67
6,65526591,How to install Python 3.5.x on Ubuntu 18.04 LT...,<p>Simple question: How to install Python 3.5....,65526611,2,1,None,2021-01-01 00:49:45.320000+00:00,NaN,2021-01-01 09:48:41.703000+00:00,NaT,None,NaN,None,6305105,None,1,0,python|ubuntu,123
7,65526474,GDB: Displaying incorrect values in struct,<p>I'm trying to implement the <code>malloc</c...,65526637,1,0,None,2021-01-01 00:16:10.673000+00:00,NaN,2021-01-01 01:02:22.447000+00:00,2021-01-01 00:39:32.250000+00:00,None,1233251.0,None,11035194,None,1,1,struct|gdb|memory-address,34
8,65526590,Vue2 + laravel6 - Component implementation,<p>I just started using Vue2 with Laravel6 and...,65526639,1,0,None,2021-01-01 00:49:32.030000+00:00,NaN,2021-01-01 01:09:45.250000+00:00,2021-01-01 01:07:22.133000+00:00,None,11960598.0,None,14605909,None,1,0,laravel|vue.js|vuejs2|laravel-6,39
9,65526522,Mobile menu css,<p>What's the best way to\nachieve going from ...,65526667,2,0,None,2021-01-01 00:30:28.633000+00:00,NaN,2021-01-01 01:46:35.280000+00:00,NaT,None,NaN,None,6787542,None,1,0,css,39


In [25]:
# Extract data for May 2021 for posts with accepted ansers
query_job = client.query(
        """
        SELECT
        id, title, body, accepted_answer_id, creation_date, favorite_count, score, tags, view_count
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-05-01'
        ORDER BY accepted_answer_id"""
    )

results = query_job.result() 
results_list = [row for row in results]
cols = ['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date', 'q_favorite_count', 'q_score', 'q_tags', 'q_view_count']
posts_questions_df = pd.DataFrame.from_records(results_list, columns=cols)
posts_questions_df.sample(n=15)

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_favorite_count,q_score,q_tags,q_view_count
594,67348377,Use variable in another function then use that...,<pre><code>def secondscene():\n second_option...,67348416,2021-05-01 16:17:23.290000+00:00,NaN,0,python,13
174852,68752284,Check if a shared library likned with -Bsymbolic,<p>How can I check if a shared library linked ...,68754380,2021-08-12 05:55:17.990000+00:00,NaN,0,linux|linker|shared-libraries,18
47066,67725700,How can I uniquely identify a specific row in ...,<p>I have got myself in a right pickle with th...,67726837,2021-05-27 15:50:37.590000+00:00,NaN,0,sql|sql-server|tsql,51
167861,68696951,Making bootstrap5 col only go down to footer,<p>For this project I have to display a specif...,68697057,2021-08-08 00:19:25.593000+00:00,NaN,-1,html|css|bootstrap-5,36
73227,67929609,IIS Web Application Allowing Anonymous Access ...,"<p>Windows Server 2012 R2, IIS 8</p>\n<p>I did...",67937548,2021-06-10 23:36:13.077000+00:00,NaN,0,authentication|iis|basic-authentication|iis-8|...,56
170284,68374703,@typescript-eslint/typedef enable call-signature,<p>I have recently upgraded my angular applica...,68716502,2021-07-14 08:36:14.543000+00:00,NaN,0,typescript|eslint,44
76002,67959594,if else sql with additional information,<p>i have a case that the new column in sql co...,67959720,2021-06-13 14:57:16.703000+00:00,NaN,0,mysql|sql|if-statement,22
131123,68402774,How can I list all the get properties from a g...,<p>Give the class:</p>\n<pre><code>export clas...,68402794,2021-07-16 02:38:06.840000+00:00,NaN,0,javascript|typescript|reflection|getter,29
79208,67985256,# of records loaded through AWS Redshift,<p>Is there a way through the AWS console to u...,67985311,2021-06-15 11:18:10.123000+00:00,NaN,1,amazon-web-services|amazon-redshift|aws-data-p...,16
145004,68459486,Log axis with matplotlib - unexpected result,<p>I'd like to create a campbell diagramm (sou...,68512885,2021-07-20 18:04:23.777000+00:00,NaN,0,python|matplotlib,58


In [26]:
# Determine data types for posts_questions_df
posts_questions_df.dtypes

q_id                                int64
q_title                            object
q_body                             object
accepted_answer_id                  int64
q_creation_date       datetime64[ns, UTC]
q_favorite_count                  float64
q_score                             int64
q_tags                             object
q_view_count                        int64
dtype: object

In [27]:
# How many missing values in the DF by column
[[column,posts_questions_df[column].isnull().sum()] for column in posts_questions_df.columns]

[['q_id', 0],
 ['q_title', 0],
 ['q_body', 0],
 ['accepted_answer_id', 0],
 ['q_creation_date', 0],
 ['q_favorite_count', 193444],
 ['q_score', 0],
 ['q_tags', 0],
 ['q_view_count', 0]]

In [28]:
len(posts_questions_df)

210368

In [29]:
# Drop q_favorite_count column because too many null values
posts_questions_df = posts_questions_df.drop(columns='q_favorite_count')
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_score', 'q_tags', 'q_view_count'],
      dtype='object')

In [30]:
# How many missing values in the DF by column
[[column,posts_questions_df[column].isnull().sum()] for column in posts_questions_df.columns]

[['q_id', 0],
 ['q_title', 0],
 ['q_body', 0],
 ['accepted_answer_id', 0],
 ['q_creation_date', 0],
 ['q_score', 0],
 ['q_tags', 0],
 ['q_view_count', 0]]

In [31]:
# Additional method to verify number of rows with zero response time (what's the possible explanation?)
q_score_explore_raw = posts_questions_df.groupby('q_score').count()
q_score_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count
q_score,,,,,,,
-11,1,1,1,1,1,1,1
-10,1,1,1,1,1,1,1
-9,6,6,6,6,6,6,6
-8,9,9,9,9,9,9,9
-7,15,15,15,15,15,15,15
...,...,...,...,...,...,...,...
103,1,1,1,1,1,1,1
125,1,1,1,1,1,1,1
153,1,1,1,1,1,1,1


In [32]:
# Bin score for more meaningful analysis
# Establish the bins.
q_score_bins = [-10, -1, 0, 10, 25, 50, 101]
q_score_group_names = ["Negative Score (<0)", "Zero Score (0)", "Low Score (0-10)", "Medium Score (10 - 25)", "High Score (25-50)", "Popular Score (50-101)"]

# Categorize score based on the bins.
posts_questions_df['q_score_tier'] = pd.cut(posts_questions_df['q_score'], q_score_bins, labels=q_score_group_names)

In [33]:
# explore scores
q_score_explore = posts_questions_df.groupby('q_score_tier').count()
q_score_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count
q_score_tier,,,,,,,,
Negative Score (<0),21235,21235,21235,21235,21235,21235,21235,21235
Zero Score (0),116565,116565,116565,116565,116565,116565,116565,116565
Low Score (0-10),72330,72330,72330,72330,72330,72330,72330,72330
Medium Score (10 - 25),169,169,169,169,169,169,169,169
High Score (25-50),46,46,46,46,46,46,46,46
Popular Score (50-101),15,15,15,15,15,15,15,15


In [34]:
# Bin score for more meaningful analysis (broader bin)

# Establish the bins.
q_score_bins = [-10, -1, 0, 101]
q_score_group_names = ["Negative Score (<0)", "Zero Score (0)", "Positive Score (>0)"]
# Categorize score based on the bins.
posts_questions_df['q_score_tier'] = pd.cut(posts_questions_df['q_score'], q_score_bins, labels=q_score_group_names)
# Bin scores
q_score_explore = posts_questions_df.groupby('q_score_tier').count()
q_score_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count
q_score_tier,,,,,,,,
Negative Score (<0),21235,21235,21235,21235,21235,21235,21235,21235
Zero Score (0),116565,116565,116565,116565,116565,116565,116565,116565
Positive Score (>0),72560,72560,72560,72560,72560,72560,72560,72560


In [35]:
# Verify whether there are redundant columns for score_tier
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_score', 'q_tags', 'q_view_count', 'q_score_tier'],
      dtype='object')

In [36]:
# Verify data in q_score_tier column
posts_questions_df.sample(n=10)

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count,q_score_tier
25777,67549640,Type Error: new() missing 1 required positiona...,<p>I am trying to implement a robot environmen...,67555588,2021-05-15 17:43:41.137000+00:00,0,python|class|typeerror,112,Zero Score (0)
58406,67817155,Button is null object reference (Android Studio),<p>I have a recyclerview and view holder to li...,67818125,2021-06-03 07:27:15.097000+00:00,0,android-studio,20,Zero Score (0)
206859,69019253,Display image dynamically on hover on title wi...,<p>I have a list of title dynamically rendered...,69021549,2021-09-01 18:57:00.260000+00:00,1,javascript|reactjs|next.js|gsap|use-ref,32,Positive Score (>0)
40271,67672797,Dropping data frames from a list based on a co...,<p>I have a list of multiple data frames and I...,67672884,2021-05-24 13:21:28.677000+00:00,1,python,26,Positive Score (>0)
161355,68640608,Azure App Configuration Managed Identity faili...,"<p>I'm following <a href=""https://i.stack.imgu...",68643614,2021-08-03 17:43:14.093000+00:00,0,c#|.net-core|azure-functions|azure-managed-ide...,32,Zero Score (0)
134409,68428639,Prop name in functional component in React,<p>I am new to React. I have some code snippet...,68428689,2021-07-18 11:31:28.033000+00:00,2,javascript|reactjs|react-native|react-props|re...,45,Positive Score (>0)
98446,68140348,Signed capabilities build failures Xcode 12,<p>I am trying to create an archive in Xcode 1...,68140706,2021-06-26 07:56:10.993000+00:00,0,ios|xcode,39,Zero Score (0)
124397,68348890,why selenium gives duplicate element?,"<p>URL = <a href=""https://www.amazon.com/gp/be...",68349042,2021-07-12 14:12:49.763000+00:00,1,python|python-3.x|selenium|selenium-webdriver|...,47,Positive Score (>0)
165210,68675011,"After every 1s, some of the boxes should chang...",<p>I have drawn some stars by using javascript...,68675388,2021-08-06 01:10:50.803000+00:00,0,javascript|jquery,48,Zero Score (0)
193397,68903791,JavaFX Circle Linear Gradient Fill by Percentage,<p>I just want to make an indicator for custom...,68906552,2021-08-24 07:56:46.850000+00:00,1,java|javafx|linear-gradients,42,Positive Score (>0)


In [37]:
# Drop score column because data transformed to the score_tier column
posts_questions_df = posts_questions_df.drop(columns='q_score')
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier'],
      dtype='object')

In [38]:
# Determine length of title
posts_questions_df['q_title_char_count']= posts_questions_df['q_title'].str.len() #character count
posts_questions_df['q_title_word_count']= posts_questions_df['q_title'].str.split().str.len() #word count
posts_questions_df.head()

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_word_count
0,67341742,Is there any type of way to call a function in...,<p>sorry in advance for my lack of technical l...,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,38,Zero Score (0),95,20
1,67341817,Produce multiple models for CVC4 SMT queries,<p>Can I get multiple models for a query like ...,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,41,Zero Score (0),44,7
2,67341895,Why does shuffling array 'A' also shuffle arra...,<p>I'm trying to make a shuffled copy of an ar...,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,50,Positive Score (>0),97,17
3,67341936,Is there a simpler way to validate user input ...,<p>I am currently learning python and I was wo...,67341961,2021-05-01 00:41:17.673000+00:00,python,57,Positive Score (>0),86,15
4,67341921,How to Install vue3 in Laravel 8 without vue-s...,<p>I have install Laravel 8 fresh project. And...,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,42,Positive Score (>0),114,17


In [39]:
# Bin by title length (char count)
q_title_explore_char = posts_questions_df.groupby('q_title_char_count').count()
q_title_explore_char

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_word_count
q_title_char_count,,,,,,,,,
15,65,65,65,65,65,65,65,65,65
16,120,120,120,120,120,120,120,120,120
17,145,145,145,145,145,145,145,145,145
18,193,193,193,193,193,193,193,193,193
19,271,271,271,271,271,271,271,271,271
...,...,...,...,...,...,...,...,...,...
146,106,106,106,106,106,106,106,106,106
147,113,113,113,113,113,113,113,113,113
148,132,132,132,132,132,132,132,132,132


In [40]:
# Bin title length by char count

# Establish the bins.
q_char_count_bins = [0, 50, 100, 150]
q_char_count_group_names = ["Short (0 - 50)", "Medium (50-100)", "Long (100-150)"]

# Categorize char length based on the bins.
posts_questions_df['q_title_char_count_bin'] = pd.cut(posts_questions_df['q_title_char_count'], q_char_count_bins, labels=q_char_count_group_names)

# Bin by char length
q_char_count_explore = posts_questions_df.groupby('q_title_char_count_bin').count()
q_char_count_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_word_count
q_title_char_count_bin,,,,,,,,,,
Short (0 - 50),81247,81247,81247,81247,81247,81247,81247,81245,81247,81247
Medium (50-100),117547,117547,117547,117547,117547,117547,117547,117542,117547,117547
Long (100-150),11574,11574,11574,11574,11574,11574,11574,11573,11574,11574


In [41]:
# Bin by title length (word count)
q_title_explore_word = posts_questions_df.groupby('q_title_word_count').count()
q_title_explore_word

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_char_count_bin
q_title_word_count,,,,,,,,,,
1,18,18,18,18,18,18,18,18,18,18
2,334,334,334,334,334,334,334,334,334,334
3,2410,2410,2410,2410,2410,2410,2410,2410,2410,2410
4,6756,6756,6756,6756,6756,6756,6756,6756,6756,6756
5,12050,12050,12050,12050,12050,12050,12050,12050,12050,12050
6,17744,17744,17744,17744,17744,17744,17744,17744,17744,17744
7,21748,21748,21748,21748,21748,21748,21748,21747,21748,21748
8,23927,23927,23927,23927,23927,23927,23927,23925,23927,23927
9,23431,23431,23431,23431,23431,23431,23431,23431,23431,23431


In [42]:
# Bin title length by word count

# Establish the bins.
q_title_word_count_bins = [0, 10, 20, 30, 40]
q_title_word_count_group_names = ["Short (0 - 10)", "Medium (10-20)", "Long (20-30)", "XL (30+)"]

# Categorize char length based on the bins.
posts_questions_df['q_title_word_count_bin'] = pd.cut(posts_questions_df['q_title_word_count'], q_title_word_count_bins, labels=q_title_word_count_group_names)

# Bin by char length
q_word_count_explore = posts_questions_df.groupby('q_title_word_count_bin').count()
q_word_count_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count,q_title_word_count,q_title_char_count_bin
q_title_word_count_bin,,,,,,,,,,,
Short (0 - 10),129640,129640,129640,129640,129640,129640,129640,129636,129640,129640,129640
Medium (10-20),76349,76349,76349,76349,76349,76349,76349,76345,76349,76349,76349
Long (20-30),4338,4338,4338,4338,4338,4338,4338,4338,4338,4338,4338
XL (30+),41,41,41,41,41,41,41,41,41,41,41


In [43]:
# Clean up posts_questions_df
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier', 'q_title_char_count',
       'q_title_word_count', 'q_title_char_count_bin',
       'q_title_word_count_bin'],
      dtype='object')

In [44]:
# Drop q_title_char_count and q_title_word_count columns because data transformed into bins
posts_questions_df = posts_questions_df.drop(columns='q_title_char_count')
posts_questions_df = posts_questions_df.drop(columns='q_title_word_count')
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier', 'q_title_char_count_bin',
       'q_title_word_count_bin'],
      dtype='object')

In [45]:
# Explore view count
q_views_explore_raw = posts_questions_df.groupby('q_view_count').count()
q_views_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count,,,,,,,,,
6,6,6,6,6,6,6,6,6,6
7,19,19,19,19,19,19,19,19,19
8,42,42,42,42,42,42,42,42,42
9,78,78,78,78,78,78,78,78,78
10,177,177,177,177,177,177,177,177,177
...,...,...,...,...,...,...,...,...,...
28307,1,1,1,1,1,1,1,1,1
33645,1,1,1,1,1,1,1,1,1
34105,1,1,1,1,1,1,1,1,1


In [46]:
q_views_explore_raw.sort_values(by=['q_id']).tail(20) # Majority views around mid 20s to 30s

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count,,,,,,,,,
42,3877,3877,3877,3877,3877,3877,3877,3877,3877
23,4027,4027,4027,4027,4027,4027,4027,4027,4027
24,4128,4128,4128,4128,4128,4128,4128,4128,4128
41,4131,4131,4131,4131,4131,4131,4131,4131,4131
40,4148,4148,4148,4148,4148,4148,4148,4148,4148
39,4317,4317,4317,4317,4317,4317,4317,4317,4317
38,4521,4521,4521,4521,4521,4521,4521,4521,4521
25,4530,4530,4530,4530,4530,4530,4530,4530,4530
37,4570,4570,4570,4570,4570,4570,4570,4570,4570


In [47]:
# Bin view count for more meaningful analysis
# Establish the bins.
q_view_count_bins = [0,  50, 500, 1000, 5000, 16000]
q_view_count_group_names = ["<50", "50-500", "500-1000", "1000-5000", "5000-16000"]

# Categorize score based on the bins.
posts_questions_df['q_view_count_bin'] = pd.cut(posts_questions_df['q_view_count'], q_view_count_bins, labels=q_view_count_group_names)

In [48]:
# Explore view count
q_views_explore = posts_questions_df.groupby('q_view_count_bin').count()
q_views_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count_bin,,,,,,,,,,
<50,140462,140462,140462,140462,140462,140462,140462,140462,140462,140462
50-500,68310,68310,68310,68310,68310,68310,68310,68308,68310,68310
500-1000,998,998,998,998,998,998,998,998,998,998
1000-5000,517,517,517,517,517,517,517,517,517,517
5000-16000,66,66,66,66,66,66,66,65,66,66


In [49]:
# Readjust bins for more distributed data - add meaningfulness to ML model inputs

# Establish the bins.
q_view_count_bins = [0, 10, 20, 30, 40, 50, 16000]
q_view_count_group_names = ["<10", "10-20", "20-30","30-40", "40-50", "50-16000"]

# Categorize score based on the bins.
posts_questions_df['q_view_count_bin'] = pd.cut(posts_questions_df['q_view_count'], q_view_count_bins, labels=q_view_count_group_names)
# Explore view count
q_views_explore = posts_questions_df.groupby('q_view_count_bin').count()
q_views_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin
q_view_count_bin,,,,,,,,,,
<10,322,322,322,322,322,322,322,322,322,322
10-20,14596,14596,14596,14596,14596,14596,14596,14596,14596,14596
20-30,43859,43859,43859,43859,43859,43859,43859,43859,43859,43859
30-40,47165,47165,47165,47165,47165,47165,47165,47165,47165,47165
40-50,34520,34520,34520,34520,34520,34520,34520,34520,34520,34520
50-16000,69891,69891,69891,69891,69891,69891,69891,69888,69891,69891


In [50]:
# Determine word count of body
posts_questions_df['q_body_word_count']= posts_questions_df['q_body'].str.split().str.len() #word count
# Group by body word count
q_body_explore_raw = posts_questions_df.groupby('q_body_word_count').count()
q_body_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin
q_body_word_count,,,,,,,,,,,
5,3,3,3,3,3,3,3,3,3,3,3
6,2,2,2,2,2,2,2,2,2,2,2
7,3,3,3,3,3,3,3,3,3,3,3
8,6,6,6,6,6,6,6,6,6,6,6
9,14,14,14,14,14,14,14,14,14,14,14
...,...,...,...,...,...,...,...,...,...,...,...
4423,1,1,1,1,1,1,1,1,1,1,1
4475,1,1,1,1,1,1,1,1,1,1,1
4762,1,1,1,1,1,1,1,1,1,1,1


In [51]:
q_body_explore_raw.sort_values(by=['q_id']).tail(20) # Majority views around mid 20s to 30s

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin
q_body_word_count,,,,,,,,,,,
104,1087,1087,1087,1087,1087,1087,1087,1087,1087,1087,1087
101,1088,1088,1088,1088,1088,1088,1088,1088,1088,1088,1088
79,1093,1093,1093,1093,1093,1093,1093,1093,1093,1093,1093
103,1095,1095,1095,1095,1095,1095,1095,1095,1095,1095,1095
85,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097
86,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097
92,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097
91,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099
95,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1101


In [52]:
# Bin body word count for more meaningful analysis
# Establish the bins.
q_body_len_bins = [0,  100, 500, 10000]
q_body_len_group_names = ["<100", "100-500", "500-10000"]

# Categorize body length based on the bins.
posts_questions_df['q_body_len_bin'] = pd.cut(posts_questions_df['q_body_word_count'], q_body_len_bins, labels=q_body_len_group_names)

# Explore body length
q_body_len_explore = posts_questions_df.groupby('q_body_len_bin').count()
q_body_len_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count
q_body_len_bin,,,,,,,,,,,,
<100,66612,66612,66612,66612,66612,66612,66612,66607,66612,66612,66603,66612
100-500,135476,135476,135476,135476,135476,135476,135476,135473,135476,135476,135470,135476
500-10000,8280,8280,8280,8280,8280,8280,8280,8280,8280,8280,8280,8280


In [53]:
# Readjust bins for more distributed data - add meaningfulness to ML model inputs

# Establish the bins.
q_body_len_bins = [0, 50, 100, 250, 500, 10000]
q_body_len_group_names = ["<50", "50-100", "100-250", "250-500", "500-10000"]

# Categorize body length based on the bins.
posts_questions_df['q_body_len_bin'] = pd.cut(posts_questions_df['q_body_word_count'], q_body_len_bins, labels=q_body_len_group_names)

# Explore body length
q_body_len_explore = posts_questions_df.groupby('q_body_len_bin').count()
q_body_len_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count
q_body_len_bin,,,,,,,,,,,,
<50,15444,15444,15444,15444,15444,15444,15444,15443,15444,15444,15443,15444
50-100,51168,51168,51168,51168,51168,51168,51168,51164,51168,51168,51160,51168
100-250,102199,102199,102199,102199,102199,102199,102199,102197,102199,102199,102193,102199
250-500,33277,33277,33277,33277,33277,33277,33277,33276,33277,33277,33277,33277
500-10000,8280,8280,8280,8280,8280,8280,8280,8280,8280,8280,8280,8280


In [54]:
# Create list with all values within q_body
#q_body = posts_questions_df['q_body']
#q_body

In [55]:
#print(q_body[47056])

In [56]:
# Find all unique HTML tags in q_body list
    # Use re library instead of hardcoding things
    # Use functions because it's a repetitive process
    
    # 1. To find all the unique tags in the q_body listand store in their own list
    # 2. Count tags within each item in q_body list (Beautiful Soup)

#test_str = q_body [47056]

#def find_tags(q_body_str):
    
    
#batRegex = re.compile(r'(<[a-z]*>)')
#m1=batRegex.search(html)
#print batRegex.findall(yourstring)




In [57]:
# Count number of each unique tag for every row in q_body list
    # For each row, how many <p>, how many <img tags>, etc
    # Store as separate column in q_body
    
# Group by tags

# Create function to count tags
    # Create list from tags column
    # Count number of "|" + 1
    
q_tags = posts_questions_df['q_tags']
q_tags # Observe the list of tags


0                        javascript|html|node.js|obs
1             smt|formal-methods|satisfiability|cvc4
2                                       python|numpy
3                                             python
4                                   laravel-8|vuejs3
                             ...                    
210363                           android-progressbar
210364                          bash|virtual-machine
210365                                 intellij-idea
210366    python|pandas|dataframe|time-series|cumsum
210367                    php|file|validation|exists
Name: q_tags, Length: 210368, dtype: object

In [58]:
q_tags_count = q_tags.str.count('\\|') + 1 # Count the number of | + 1 to get the number of tags in each row

posts_questions_df['q_tags_count'] = q_tags_count # Append the list as a column to the posts_questions_df 
tag_count = posts_questions_df[['q_tags','q_tags_count']]
tag_count

,q_tags,q_tags_count
0,javascript|html|node.js|obs,4
1,smt|formal-methods|satisfiability|cvc4,4
2,python|numpy,2
3,python,1
4,laravel-8|vuejs3,2
...,...,...
210363,android-progressbar,1
210364,bash|virtual-machine,2
210365,intellij-idea,1
210366,python|pandas|dataframe|time-series|cumsum,5


In [59]:
# Group by tag count
tag_count_exlore = tag_count.groupby('q_tags_count').count()
tag_count_exlore

,q_tags
q_tags_count,
1,24799
2,55254
3,58890
4,40495
5,30930


In [60]:
# Expand our query beyond May 2021, go back 12 months

# Create our own tags table and categorize and encode it

In [61]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.posts_answers`
        WHERE creation_date > '2021-05-01'
        ORDER BY id
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]
cols = [field.name for field in results.schema]
posts_answers = pd.DataFrame.from_records(results_list, columns=cols)
posts_answers

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,67341731,None,<p>This call is causing it:</p>\n<pre><code>so...,None,None,0,None,2021-05-01 00:00:43.057000+00:00,None,2021-05-01 00:00:43.057000+00:00,NaT,None,NaN,None,1980846,67311357,2,0,None,None
1,67341735,None,<p>It looks like you broke your <code>.vimrc</...,None,None,3,None,2021-05-01 00:02:14.853000+00:00,None,2021-05-01 00:02:14.853000+00:00,NaT,None,NaN,None,3251281,67341681,2,1,None,None
2,67341736,None,<p>After extensive reading of the VBA document...,None,None,0,None,2021-05-01 00:02:18.417000+00:00,None,2021-05-01 00:02:18.417000+00:00,NaT,None,NaN,None,10967640,67326302,2,0,None,None
3,67341740,None,"<p>The double bar is the distance measure, and...",None,None,0,None,2021-05-01 00:02:59.713000+00:00,None,2021-05-01 00:02:59.713000+00:00,NaT,None,NaN,None,15804985,44038581,2,0,None,None
4,67341741,None,<p>Long characters continue to be a problem fo...,None,None,0,None,2021-05-01 00:03:03.637000+00:00,None,2021-05-05 19:39:39.557000+00:00,2021-05-05 19:39:39.557000+00:00,None,1104179.0,None,1104179,48127045,2,1,None,None
5,67341744,None,<pre><code>**Do you mean so?**\nprivate var it...,None,None,4,None,2021-05-01 00:03:39.260000+00:00,None,2021-05-01 00:03:39.260000+00:00,NaT,None,NaN,None,14893497,67341386,2,0,None,None
6,67341745,None,<p>The file could be sent to the current worki...,None,None,0,None,2021-05-01 00:03:47.920000+00:00,None,2021-05-01 00:03:47.920000+00:00,NaT,None,NaN,None,12370687,67336278,2,0,None,None
7,67341746,None,<p>First of all this is not gonna be a technic...,None,None,0,None,2021-05-01 00:03:56.097000+00:00,None,2021-05-01 00:03:56.097000+00:00,NaT,None,NaN,None,15125036,67341278,2,-1,None,None
8,67341748,None,<p>Working nginx config:</p>\n<pre><code>upstr...,None,None,0,None,2021-05-01 00:04:10.380000+00:00,None,2021-05-01 00:04:10.380000+00:00,NaT,None,NaN,None,798502,67312468,2,0,None,None
9,67341749,None,<p>I was able to build appbundle by running in...,None,None,2,None,2021-05-01 00:04:10.487000+00:00,None,2021-05-01 03:58:09.633000+00:00,2021-05-01 03:58:09.633000+00:00,None,2628868.0,None,15120078,67224299,2,2,None,None


In [62]:
# Pull only relevant columns to ML model
query_job = client.query(
        """
        SELECT
        id, creation_date, parent_id
        FROM `bigquery-public-data.stackoverflow.posts_answers`
        WHERE creation_date > '2021-05-01'
        ORDER BY id"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]
cols = ['a_id', 'a_creation_date', 'q_id']
posts_answers_df = pd.DataFrame.from_records(results_list, columns=cols)
posts_answers_df

,a_id,a_creation_date,q_id
0,67341731,2021-05-01 00:00:43.057000+00:00,67311357
1,67341735,2021-05-01 00:02:14.853000+00:00,67341681
2,67341736,2021-05-01 00:02:18.417000+00:00,67326302
3,67341740,2021-05-01 00:02:59.713000+00:00,44038581
4,67341741,2021-05-01 00:03:03.637000+00:00,48127045
...,...,...,...
698795,69061022,2021-09-05 06:21:38.487000+00:00,69060415
698796,69061025,2021-09-05 06:22:13.693000+00:00,69059807
698797,69061028,2021-09-05 06:22:52.787000+00:00,69059980
698798,69061029,2021-09-05 06:23:42.780000+00:00,69060590


In [63]:
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier', 'q_title_char_count_bin',
       'q_title_word_count_bin', 'q_view_count_bin', 'q_body_word_count',
       'q_body_len_bin', 'q_tags_count'],
      dtype='object')

In [64]:
#post_questions = post_questions.rename(columns={"id":"parent_id"})
#post_questions

In [65]:
duration_df = pd.merge(posts_questions_df, posts_answers_df, left_on=  ['accepted_answer_id'],
                   right_on= ['a_id'], how = 'left', suffixes=['_pq','_pa'])
duration_df

,q_id_pq,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,a_id,a_creation_date,q_id_pa
0,67341742,Is there any type of way to call a function in...,<p>sorry in advance for my lack of technical l...,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,38,Zero Score (0),Medium (50-100),Medium (10-20),30-40,217,100-250,4,67341801,2021-05-01 00:14:38.340000+00:00,67341742
1,67341817,Produce multiple models for CVC4 SMT queries,<p>Can I get multiple models for a query like ...,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,41,Zero Score (0),Short (0 - 50),Short (0 - 10),40-50,47,<50,4,67341857,2021-05-01 00:26:39.053000+00:00,67341817
2,67341895,Why does shuffling array 'A' also shuffle arra...,<p>I'm trying to make a shuffled copy of an ar...,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,50,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,423,250-500,2,67341911,2021-05-01 00:36:54.910000+00:00,67341895
3,67341936,Is there a simpler way to validate user input ...,<p>I am currently learning python and I was wo...,67341961,2021-05-01 00:41:17.673000+00:00,python,57,Positive Score (>0),Medium (50-100),Medium (10-20),50-16000,243,100-250,1,67341961,2021-05-01 00:46:37.893000+00:00,67341936
4,67341921,How to Install vue3 in Laravel 8 without vue-s...,<p>I have install Laravel 8 fresh project. And...,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,42,Positive Score (>0),Long (100-150),Medium (10-20),40-50,57,50-100,2,67341974,2021-05-01 00:48:52.390000+00:00,67341921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210363,69060773,android studio: center ProgressPar on top of R...,<p>I'm working on a chat app where users can s...,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar,7,Zero Score (0),Medium (50-100),Short (0 - 10),<10,269,250-500,1,69060817,2021-09-05 05:39:44.423000+00:00,69060773
210364,69060793,"bash command for hostname only returns ""server...",<p>I'm very new to bash scripting and I'm stuc...,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine,13,Positive Score (>0),Medium (50-100),Medium (10-20),10-20,164,100-250,2,69060824,2021-09-05 05:41:47.300000+00:00,69060793
210365,69060850,Intellij how to delete all line containing ann...,<p>I want to delete all lines that contains an...,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea,11,Positive Score (>0),Medium (50-100),Short (0 - 10),10-20,46,<50,1,69060862,2021-09-05 05:51:12.690000+00:00,69060850
210366,69060801,Getting sum data for smoothly shifting groups ...,<p>I have a time series data of the following ...,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum,28,Positive Score (>0),Medium (50-100),Medium (10-20),20-30,255,250-500,5,69060898,2021-09-05 05:58:50.680000+00:00,69060801


In [66]:
duration_df.drop(columns=['q_title', 'q_body', 'q_tags', 'q_view_count', 'q_score_tier', 'q_title_char_count_bin',
       'q_title_word_count_bin', 'q_view_count_bin', 'q_body_word_count',
       'q_body_len_bin', 'q_tags_count'], inplace=True)
duration_df

,q_id_pq,accepted_answer_id,q_creation_date,a_id,a_creation_date,q_id_pa
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,67341801,2021-05-01 00:14:38.340000+00:00,67341742
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,67341857,2021-05-01 00:26:39.053000+00:00,67341817
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,67341911,2021-05-01 00:36:54.910000+00:00,67341895
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,67341961,2021-05-01 00:46:37.893000+00:00,67341936
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,67341974,2021-05-01 00:48:52.390000+00:00,67341921
...,...,...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,69060817,2021-09-05 05:39:44.423000+00:00,69060773
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,69060824,2021-09-05 05:41:47.300000+00:00,69060793
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,69060862,2021-09-05 05:51:12.690000+00:00,69060850
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,69060898,2021-09-05 05:58:50.680000+00:00,69060801


In [67]:
# Compare accepted_answer_id and answer_id to verify identical
duration_df['accepted_answer_id'].equals(duration_df['a_id'])

True

In [68]:
# Drop "answer_id" column because redundant
duration_df = duration_df.drop(columns=['a_id', 'q_id_pa'])
duration_df = duration_df.rename(columns={"q_id_pq":"q_id"})
duration_df

,q_id,accepted_answer_id,q_creation_date,a_creation_date
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,2021-05-01 00:14:38.340000+00:00
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,2021-05-01 00:26:39.053000+00:00
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,2021-05-01 00:36:54.910000+00:00
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,2021-05-01 00:46:37.893000+00:00
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,2021-05-01 00:48:52.390000+00:00
...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,2021-09-05 05:39:44.423000+00:00
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,2021-09-05 05:41:47.300000+00:00
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,2021-09-05 05:51:12.690000+00:00
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,2021-09-05 05:58:50.680000+00:00


In [69]:
# Add column for day of question_creation_date [question_day]
duration_df['q_day'] = duration_df['q_creation_date'].dt.day_name()
duration_df

,q_id,accepted_answer_id,q_creation_date,a_creation_date,q_day
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,2021-05-01 00:14:38.340000+00:00,Saturday
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,2021-05-01 00:26:39.053000+00:00,Saturday
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,2021-05-01 00:36:54.910000+00:00,Saturday
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,2021-05-01 00:46:37.893000+00:00,Saturday
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,2021-05-01 00:48:52.390000+00:00,Saturday
...,...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,2021-09-05 05:39:44.423000+00:00,Sunday
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,2021-09-05 05:41:47.300000+00:00,Sunday
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,2021-09-05 05:51:12.690000+00:00,Sunday
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,2021-09-05 05:58:50.680000+00:00,Sunday


In [70]:
# Add column for hour value of question_creation_date [question_time]
duration_df['q_hour'] = duration_df['q_creation_date'].dt.hour
duration_df['q_hour_min'] = duration_df['q_creation_date'].dt.strftime('%H:%M')

duration_df

,q_id,accepted_answer_id,q_creation_date,a_creation_date,q_day,q_hour,q_hour_min
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,2021-05-01 00:14:38.340000+00:00,Saturday,0,00:03
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,2021-05-01 00:26:39.053000+00:00,Saturday,0,00:17
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,2021-05-01 00:36:54.910000+00:00,Saturday,0,00:32
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,2021-05-01 00:46:37.893000+00:00,Saturday,0,00:41
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,2021-05-01 00:48:52.390000+00:00,Saturday,0,00:38
...,...,...,...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,2021-09-05 05:39:44.423000+00:00,Sunday,5,05:31
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,2021-09-05 05:41:47.300000+00:00,Sunday,5,05:34
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,2021-09-05 05:51:12.690000+00:00,Sunday,5,05:45
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,2021-09-05 05:58:50.680000+00:00,Sunday,5,05:36


In [71]:
# Add column for duration till accepted answer (answer_creation_date - question_creation_date)
duration_df['accepted_answer_duration'] = (duration_df['a_creation_date']-duration_df['q_creation_date'])/np.timedelta64(1,'h')
duration_df = duration_df.sort_values(by='accepted_answer_duration', ascending=False)
duration_df

,q_id,accepted_answer_id,q_creation_date,a_creation_date,q_day,q_hour,q_hour_min,accepted_answer_duration
207957,67420325,69033254,2021-05-06 14:36:35.883000+00:00,2021-09-02 15:34:47.857000+00:00,Thursday,14,14:36,2856.969993
202268,67365624,68981014,2021-05-03 08:18:01.683000+00:00,2021-08-30 08:27:48.440000+00:00,Monday,8,08:18,2856.162988
210268,67454378,69059067,2021-05-09 04:49:57.043000+00:00,2021-09-04 21:28:51.283000+00:00,Sunday,4,04:49,2848.648400
196437,67355370,68931282,2021-05-02 10:37:24.760000+00:00,2021-08-26 00:54:18.303000+00:00,Sunday,10,10:37,2774.281540
198903,67385222,68952849,2021-05-04 12:57:32.417000+00:00,2021-08-27 11:43:25.663000+00:00,Tuesday,12,12:57,2758.764791
...,...,...,...,...,...,...,...,...
113654,68262350,68262351,2021-07-05 21:41:04.843000+00:00,2021-07-05 21:41:04.843000+00:00,Monday,21,21:41,0.000000
187252,68855357,68855358,2021-08-19 23:10:29.470000+00:00,2021-08-19 23:10:29.470000+00:00,Thursday,23,23:10,0.000000
195492,68923827,68923828,2021-08-25 13:27:06.257000+00:00,2021-08-25 13:27:06.257000+00:00,Wednesday,13,13:27,0.000000
44851,67708855,67708856,2021-05-26 16:14:40.067000+00:00,2021-05-26 16:14:40.067000+00:00,Wednesday,16,16:14,0.000000


In [72]:
ML_input_df = pd.merge(posts_questions_df, duration_df, left_on=  ['q_id'],
                   right_on= ['q_id'], how = 'left', suffixes=['_pq','_dur'])
ML_input_df

,q_id,q_title,q_body,accepted_answer_id_pq,q_creation_date_pq,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,...,q_body_word_count,q_body_len_bin,q_tags_count,accepted_answer_id_dur,q_creation_date_dur,a_creation_date,q_day,q_hour,q_hour_min,accepted_answer_duration
0,67341742,Is there any type of way to call a function in...,<p>sorry in advance for my lack of technical l...,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,38,Zero Score (0),Medium (50-100),Medium (10-20),...,217,100-250,4,67341801,2021-05-01 00:03:03.923000+00:00,2021-05-01 00:14:38.340000+00:00,Saturday,0,00:03,0.192894
1,67341817,Produce multiple models for CVC4 SMT queries,<p>Can I get multiple models for a query like ...,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,41,Zero Score (0),Short (0 - 50),Short (0 - 10),...,47,<50,4,67341857,2021-05-01 00:17:39.380000+00:00,2021-05-01 00:26:39.053000+00:00,Saturday,0,00:17,0.149909
2,67341895,Why does shuffling array 'A' also shuffle arra...,<p>I'm trying to make a shuffled copy of an ar...,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,50,Positive Score (>0),Medium (50-100),Medium (10-20),...,423,250-500,2,67341911,2021-05-01 00:32:18.187000+00:00,2021-05-01 00:36:54.910000+00:00,Saturday,0,00:32,0.076868
3,67341936,Is there a simpler way to validate user input ...,<p>I am currently learning python and I was wo...,67341961,2021-05-01 00:41:17.673000+00:00,python,57,Positive Score (>0),Medium (50-100),Medium (10-20),...,243,100-250,1,67341961,2021-05-01 00:41:17.673000+00:00,2021-05-01 00:46:37.893000+00:00,Saturday,0,00:41,0.088950
4,67341921,How to Install vue3 in Laravel 8 without vue-s...,<p>I have install Laravel 8 fresh project. And...,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,42,Positive Score (>0),Long (100-150),Medium (10-20),...,57,50-100,2,67341974,2021-05-01 00:38:52.743000+00:00,2021-05-01 00:48:52.390000+00:00,Saturday,0,00:38,0.166569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210363,69060773,android studio: center ProgressPar on top of R...,<p>I'm working on a chat app where users can s...,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar,7,Zero Score (0),Medium (50-100),Short (0 - 10),...,269,250-500,1,69060817,2021-09-05 05:31:01.447000+00:00,2021-09-05 05:39:44.423000+00:00,Sunday,5,05:31,0.145271
210364,69060793,"bash command for hostname only returns ""server...",<p>I'm very new to bash scripting and I'm stuc...,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine,13,Positive Score (>0),Medium (50-100),Medium (10-20),...,164,100-250,2,69060824,2021-09-05 05:34:58.063000+00:00,2021-09-05 05:41:47.300000+00:00,Sunday,5,05:34,0.113677
210365,69060850,Intellij how to delete all line containing ann...,<p>I want to delete all lines that contains an...,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea,11,Positive Score (>0),Medium (50-100),Short (0 - 10),...,46,<50,1,69060862,2021-09-05 05:45:41.713000+00:00,2021-09-05 05:51:12.690000+00:00,Sunday,5,05:45,0.091938
210366,69060801,Getting sum data for smoothly shifting groups ...,<p>I have a time series data of the following ...,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum,28,Positive Score (>0),Medium (50-100),Medium (10-20),...,255,250-500,5,69060898,2021-09-05 05:36:53.920000+00:00,2021-09-05 05:58:50.680000+00:00,Sunday,5,05:36,0.365767


In [73]:
# Additional method to verify number of rows with zero response time (what's the possible explanation?)
zero_time = duration_df.groupby('accepted_answer_duration').count()
zero_time 

,q_id,accepted_answer_id,q_creation_date,a_creation_date,q_day,q_hour,q_hour_min
accepted_answer_duration,,,,,,,
0.000000,930,930,930,930,930,930,930
0.000166,1,1,1,1,1,1,1
0.000191,1,1,1,1,1,1,1
0.000194,1,1,1,1,1,1,1
0.003704,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
2758.764791,1,1,1,1,1,1,1
2774.281540,1,1,1,1,1,1,1
2848.648400,1,1,1,1,1,1,1


In [74]:
# drop data rows where duration is 0, because assumped glitch
indexNames = ML_input_df[ML_input_df['accepted_answer_duration'] == 0.000000 ].index
ML_input_df.drop(indexNames, inplace=True)
ML_input_df

,q_id,q_title,q_body,accepted_answer_id_pq,q_creation_date_pq,q_tags,q_view_count,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,...,q_body_word_count,q_body_len_bin,q_tags_count,accepted_answer_id_dur,q_creation_date_dur,a_creation_date,q_day,q_hour,q_hour_min,accepted_answer_duration
0,67341742,Is there any type of way to call a function in...,<p>sorry in advance for my lack of technical l...,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,38,Zero Score (0),Medium (50-100),Medium (10-20),...,217,100-250,4,67341801,2021-05-01 00:03:03.923000+00:00,2021-05-01 00:14:38.340000+00:00,Saturday,0,00:03,0.192894
1,67341817,Produce multiple models for CVC4 SMT queries,<p>Can I get multiple models for a query like ...,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,41,Zero Score (0),Short (0 - 50),Short (0 - 10),...,47,<50,4,67341857,2021-05-01 00:17:39.380000+00:00,2021-05-01 00:26:39.053000+00:00,Saturday,0,00:17,0.149909
2,67341895,Why does shuffling array 'A' also shuffle arra...,<p>I'm trying to make a shuffled copy of an ar...,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,50,Positive Score (>0),Medium (50-100),Medium (10-20),...,423,250-500,2,67341911,2021-05-01 00:32:18.187000+00:00,2021-05-01 00:36:54.910000+00:00,Saturday,0,00:32,0.076868
3,67341936,Is there a simpler way to validate user input ...,<p>I am currently learning python and I was wo...,67341961,2021-05-01 00:41:17.673000+00:00,python,57,Positive Score (>0),Medium (50-100),Medium (10-20),...,243,100-250,1,67341961,2021-05-01 00:41:17.673000+00:00,2021-05-01 00:46:37.893000+00:00,Saturday,0,00:41,0.088950
4,67341921,How to Install vue3 in Laravel 8 without vue-s...,<p>I have install Laravel 8 fresh project. And...,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,42,Positive Score (>0),Long (100-150),Medium (10-20),...,57,50-100,2,67341974,2021-05-01 00:38:52.743000+00:00,2021-05-01 00:48:52.390000+00:00,Saturday,0,00:38,0.166569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210363,69060773,android studio: center ProgressPar on top of R...,<p>I'm working on a chat app where users can s...,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar,7,Zero Score (0),Medium (50-100),Short (0 - 10),...,269,250-500,1,69060817,2021-09-05 05:31:01.447000+00:00,2021-09-05 05:39:44.423000+00:00,Sunday,5,05:31,0.145271
210364,69060793,"bash command for hostname only returns ""server...",<p>I'm very new to bash scripting and I'm stuc...,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine,13,Positive Score (>0),Medium (50-100),Medium (10-20),...,164,100-250,2,69060824,2021-09-05 05:34:58.063000+00:00,2021-09-05 05:41:47.300000+00:00,Sunday,5,05:34,0.113677
210365,69060850,Intellij how to delete all line containing ann...,<p>I want to delete all lines that contains an...,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea,11,Positive Score (>0),Medium (50-100),Short (0 - 10),...,46,<50,1,69060862,2021-09-05 05:45:41.713000+00:00,2021-09-05 05:51:12.690000+00:00,Sunday,5,05:45,0.091938
210366,69060801,Getting sum data for smoothly shifting groups ...,<p>I have a time series data of the following ...,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum,28,Positive Score (>0),Medium (50-100),Medium (10-20),...,255,250-500,5,69060898,2021-09-05 05:36:53.920000+00:00,2021-09-05 05:58:50.680000+00:00,Sunday,5,05:36,0.365767


In [75]:
ML_input_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id_pq',
       'q_creation_date_pq', 'q_tags', 'q_view_count', 'q_score_tier',
       'q_title_char_count_bin', 'q_title_word_count_bin', 'q_view_count_bin',
       'q_body_word_count', 'q_body_len_bin', 'q_tags_count',
       'accepted_answer_id_dur', 'q_creation_date_dur', 'a_creation_date',
       'q_day', 'q_hour', 'q_hour_min', 'accepted_answer_duration'],
      dtype='object')

In [76]:
ML_input_df.drop(columns=['q_title', 'q_body', 'accepted_answer_id_pq',
       'q_creation_date_pq', 'q_tags', 'q_view_count', 'q_creation_date_dur', 'a_creation_date'], inplace=True)
ML_input_df = ML_input_df.rename(columns={"accepted_answer_id_dur":"accepted_answer_id"})
ML_input_df

,q_id,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,accepted_answer_id,q_day,q_hour,q_hour_min,accepted_answer_duration
0,67341742,Zero Score (0),Medium (50-100),Medium (10-20),30-40,217,100-250,4,67341801,Saturday,0,00:03,0.192894
1,67341817,Zero Score (0),Short (0 - 50),Short (0 - 10),40-50,47,<50,4,67341857,Saturday,0,00:17,0.149909
2,67341895,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,423,250-500,2,67341911,Saturday,0,00:32,0.076868
3,67341936,Positive Score (>0),Medium (50-100),Medium (10-20),50-16000,243,100-250,1,67341961,Saturday,0,00:41,0.088950
4,67341921,Positive Score (>0),Long (100-150),Medium (10-20),40-50,57,50-100,2,67341974,Saturday,0,00:38,0.166569
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210363,69060773,Zero Score (0),Medium (50-100),Short (0 - 10),<10,269,250-500,1,69060817,Sunday,5,05:31,0.145271
210364,69060793,Positive Score (>0),Medium (50-100),Medium (10-20),10-20,164,100-250,2,69060824,Sunday,5,05:34,0.113677
210365,69060850,Positive Score (>0),Medium (50-100),Short (0 - 10),10-20,46,<50,1,69060862,Sunday,5,05:45,0.091938
210366,69060801,Positive Score (>0),Medium (50-100),Medium (10-20),20-30,255,250-500,5,69060898,Sunday,5,05:36,0.365767


In [130]:
ML_input_df['accepted_answer_duration']=ML_input_df['accepted_answer_duration'].dropna()
ML_input_df

,q_id,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,accepted_answer_id,q_day,q_hour,q_hour_min,accepted_answer_duration,accepted_answe_duration
0,67341742,Zero Score (0),Medium (50-100),Medium (10-20),30-40,217,100-250,4,67341801,Saturday,0,00:03,0.192894,0.192894
1,67341817,Zero Score (0),Short (0 - 50),Short (0 - 10),40-50,47,<50,4,67341857,Saturday,0,00:17,0.149909,0.149909
2,67341895,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,423,250-500,2,67341911,Saturday,0,00:32,0.076868,0.076868
3,67341936,Positive Score (>0),Medium (50-100),Medium (10-20),50-16000,243,100-250,1,67341961,Saturday,0,00:41,0.088950,0.088950
4,67341921,Positive Score (>0),Long (100-150),Medium (10-20),40-50,57,50-100,2,67341974,Saturday,0,00:38,0.166569,0.166569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210363,69060773,Zero Score (0),Medium (50-100),Short (0 - 10),<10,269,250-500,1,69060817,Sunday,5,05:31,0.145271,0.145271
210364,69060793,Positive Score (>0),Medium (50-100),Medium (10-20),10-20,164,100-250,2,69060824,Sunday,5,05:34,0.113677,0.113677
210365,69060850,Positive Score (>0),Medium (50-100),Short (0 - 10),10-20,46,<50,1,69060862,Sunday,5,05:45,0.091938,0.091938
210366,69060801,Positive Score (>0),Medium (50-100),Medium (10-20),20-30,255,250-500,5,69060898,Sunday,5,05:36,0.365767,0.365767


In [131]:
ML_input_df.dropna()

,q_id,q_score_tier,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count,q_body_len_bin,q_tags_count,accepted_answer_id,q_day,q_hour,q_hour_min,accepted_answer_duration,accepted_answe_duration
0,67341742,Zero Score (0),Medium (50-100),Medium (10-20),30-40,217,100-250,4,67341801,Saturday,0,00:03,0.192894,0.192894
1,67341817,Zero Score (0),Short (0 - 50),Short (0 - 10),40-50,47,<50,4,67341857,Saturday,0,00:17,0.149909,0.149909
2,67341895,Positive Score (>0),Medium (50-100),Medium (10-20),40-50,423,250-500,2,67341911,Saturday,0,00:32,0.076868,0.076868
3,67341936,Positive Score (>0),Medium (50-100),Medium (10-20),50-16000,243,100-250,1,67341961,Saturday,0,00:41,0.088950,0.088950
4,67341921,Positive Score (>0),Long (100-150),Medium (10-20),40-50,57,50-100,2,67341974,Saturday,0,00:38,0.166569,0.166569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210363,69060773,Zero Score (0),Medium (50-100),Short (0 - 10),<10,269,250-500,1,69060817,Sunday,5,05:31,0.145271,0.145271
210364,69060793,Positive Score (>0),Medium (50-100),Medium (10-20),10-20,164,100-250,2,69060824,Sunday,5,05:34,0.113677,0.113677
210365,69060850,Positive Score (>0),Medium (50-100),Short (0 - 10),10-20,46,<50,1,69060862,Sunday,5,05:45,0.091938,0.091938
210366,69060801,Positive Score (>0),Medium (50-100),Medium (10-20),20-30,255,250-500,5,69060898,Sunday,5,05:36,0.365767,0.365767


## Linear Regression Model

In [134]:
## pseudo code
X = ML_input_df[['q_body_word_count','q_tags_count','q_hour']]
y = ML_input_df['accepted_answer_duration']
X.head()
print("Shape: ", X.shape, y.shape)

Shape:  (209438, 3) (209438,)


In [135]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X,y)

LinearRegression()

In [136]:
 from sklearn.metrics import mean_squared_error, r2_score

# Use model to predict a value
predicted = model.predict(X)

# Score the prediction with MSE and R2
mse = mean_squared_error(y, predicted)
r2 = r2_score(y, predicted)

print(f"mean squared error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

mean squared error (MSE): 14122.88194406744
R-squared (R2 ): 0.0032075565054413557
